In [1]:
from utils import *
import numpy as np
import pickle
# import torch

In [2]:
index_to_english_alphabet=pickle.load(open('vocab_tools/index_to_english_alphabet.pickle', 'rb'))
index_to_hindi_alphabet=pickle.load(open('vocab_tools/index_to_hindi_alphabet.pickle', 'rb'))

In [3]:
hindi_alphabet_to_index=pickle.load(open('vocab_tools/hindi_alphabet_to_index.pickle', 'rb')) 
english_alphabet_to_index=pickle.load(open('vocab_tools/english_alphabet_to_index.pickle', 'rb')) 

In [4]:
X_train=np.load('simple_data/X_train.npy')
X_valid=np.load('simple_data/X_val.npy')

y_train=np.load('simple_data/y_train.npy')
y_valid=np.load('simple_data/y_val.npy')

In [5]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/opt/anaconda/envs/shared_conda_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Eng_Hind_Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, in_file, out_file, root_dir='simple_data',device='cuda'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.input = torch.tensor(np.load(root_dir+'/'+in_file))
        self.output = torch.tensor(np.load(root_dir+'/'+out_file))
        
        assert(len(self.input)==len(self.output),"Error: I/O Lengths must be same")
        

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X=self.input[idx]
        X=X.to(device)
        y=self.output[idx]
        y=y.to(device)
        


        sample = {'input': X, 'output': y}

        return sample

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

In [9]:
training_data=Eng_Hind_Dataset("X_train.npy","y_train.npy",device=device)
val_data=Eng_Hind_Dataset("X_val.npy","y_val.npy",device=device)
test_data=Eng_Hind_Dataset("X_test.npy","y_test.npy",device=device)

In [10]:
train_dataloader = DataLoader(training_data, batch_size=16,shuffle=True)

In [11]:
val_dataloader = DataLoader(val_data, batch_size=16,shuffle=True)

In [13]:
aaa=next(iter(train_dataloader))

In [17]:
aaa['input'].shape

torch.Size([16, 30])

In [12]:
import torch.nn as nn

In [13]:
def cell_type(mode:str='rnn'):
    mode=mode.lower()
    if mode == 'rnn':
        return nn.RNN
    elif mode =='gru':
        return nn.GRU
    else:
        return nn.LSTM
        

In [14]:
class Encoder(nn.Module):
    """
    Input :
        - source batch
    Layer : 
        source batch -> Embedding -> LSTM
    Output :
        - LSTM hidden state
        - LSTM cell state

    Parmeters
    ---------
    input_dim : int
        Input dimension, should equal to the source vocab size.
    
    emb_dim : int
        Embedding layer's dimension.
        
    hid_dim : int
        LSTM Hidden/Cell state's dimension.
        
    n_layers : int
        Number of LSTM layers.
        
    dropout : float
        Dropout for the LSTM layer.
    """

    def __init__(self, input_size, embed_size, hid_size, num_layers, cell_mode, dropout, is_bi):
        super().__init__()
        
        # create embedding layer
        self.embedding = nn.Embedding(input_size, embed_size,padding_idx=english_alphabet_to_index['.'])

        #creating LSTM/GRU/RNN cell
        cell=cell_type(cell_mode)
        
        self.cell=cell(embed_size,hid_size,num_layers,dropout=dropout,bidirectional=is_bi,batch_first=True)
        self.cell_mode=cell_mode
        
        

    def forward(self, input_batch: torch.LongTensor):
        """

        Parameters
        ----------
        src_batch : 2d torch.LongTensor
            Batched tokenized source sentence of shape [sent len, batch size].

        Returns
        -------
        hidden, cell : 3d torch.LongTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]
        """
        embedded = self.embedding(input_batch) # [sent len, batch size, emb dim]
        #print('encoder embd',embedded.shape)
        
        if self.cell_mode.lower()=='lstm':
            outputs, (hidden, cell) = self.cell(embedded)
            
            
        else:
            outputs, hidden = self.cell(embedded)
            cell=outputs
        # outputs -> [sent len, batch size, hidden dim * n directions]
        return hidden, cell

In [15]:
class Decoder(nn.Module):
    """
    Input :
        - first token in the target batch
        - LSTM hidden state from the encoder
        - LSTM cell state from the encoder
    Layer :
        target batch -> Embedding -- 
                                   |
        encoder hidden state ------|--> LSTM -> Linear
                                   |
        encoder cell state   -------
        
    Output :
        - prediction
        - LSTM hidden state
        - LSTM cell state

    Parmeters
    ---------
    output : int
        Output dimension, should equal to the target vocab size.
    
    emb_dim : int
        Embedding layer's dimension.
        
    hid_dim : int
        LSTM Hidden/Cell state's dimension.
        
    n_layers : int
        Number of LSTM layers.
        
    dropout : float
        Dropout for the LSTM layer.
    """

    
    def __init__(self, output_size, embed_size, hid_size, num_layers, cell_mode, dropout, is_bi):
        super().__init__()


        self.embedding = nn.Embedding(output_size, embed_size,padding_idx=hindi_alphabet_to_index['.'])
        
        cell=cell_type(cell_mode)
        
        self.cell=cell(embed_size,hid_size,num_layers,dropout=dropout,bidirectional=is_bi,batch_first=True)
        self.out = nn.Linear(hid_size, output_size)
        
        self.output_size=output_size
        self.cell_mode=cell_mode
        

    def forward(self, trg: torch.LongTensor, hidden: torch.FloatTensor, cell: torch.FloatTensor):
        """

        Parameters
        ----------
        trg : 1d torch.LongTensor
            Batched tokenized source sentence of shape [batch size].
            
        hidden, cell : 3d torch.FloatTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]

        Returns
        -------
        prediction : 2d torch.LongTensor
            For each token in the batch, the predicted target vobulary.
            Shape [batch size, output dim]

        hidden, cell : 3d torch.FloatTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]
        """
        # [1, batch size, emb dim], the 1 serves as sent len
        embedded = self.embedding(trg.unsqueeze(1))
        if self.cell_mode.lower()=='lstm':
            #print('decoder embed',embedded.shape)
            outputs, (hidden, cell) = self.cell(embedded, (hidden, cell))
        else:
            outputs, hidden = self.cell(embedded, hidden)
            cell=hidden
        prediction = self.out(outputs.squeeze(1))
        return prediction, hidden, cell

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, source_batch, target_batch, teacher_forcing_ratio=0.5):

        batch_size,max_len  = target_batch.shape
        #print(max_len,batch_size)
        target_vocab_size = self.decoder.output_size
        #print(target_vocab_size)

        # tensor to store decoder's output
        outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)

        # last hidden & cell state of the encoder is used as the decoder's initial hidden state
        hidden, cell = self.encoder(source_batch)       

        trg = target_batch[:,0]
        for i in range(1, max_len):
            prediction, hidden, cell = self.decoder(trg, hidden, cell)
            outputs[i] = prediction

            if random.random() < teacher_forcing_ratio:
                trg = target_batch[:,i]
            else:
                trg = prediction.argmax(1)

        return outputs


        
        

In [17]:
hindi_alphabet_to_index['>']

1

In [18]:
ss1=next(iter(train_dataloader))

In [19]:
ss1

{'input': tensor([[ 0, 12, 17, 11, 11,  9, 20,  8, 18,  4,  5,  1,  2,  2,  2,  2,  2,  2,
           2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
         [ 0,  5,  4,  5,  8,  9,  5, 16, 17,  1,  2,  2,  2,  2,  2,  2,  2,  2,
           2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
         [ 0,  6,  8,  5,  9,  3, 14, 23,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,
           2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
         [ 0, 19,  9, 20,  9,  6,  7,  9,  5, 23,  9,  1,  2,  2,  2,  2,  2,  2,
           2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
         [ 0,  9, 20,  4, 11, 27,  9,  6, 14,  1,  2,  2,  2,  2,  2,  2,  2,  2,
           2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
         [ 0,  3,  9, 20,  3,  7,  9,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
           2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
         [ 0, 12,  4, 11, 23, 14, 10,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
           2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
    

In [19]:
def accuracy_calc(target_seq,seq2,mode='full',device=device):# predicted
    eos_index=(target_seq==hindi_alphabet_to_index['>']).nonzero()
    eos_idx=eos_index[:,1]
    
    correct=torch.Tensor([0]).to(device)
    correct_chars=torch.Tensor([0]).to(device)
    tot_chars=torch.Tensor([0]).to(device)
    for iterate,idx in enumerate(eos_idx):
        inputter=seq2[iterate][:idx]
        outputter=target_seq[iterate][:idx]
        if torch.all(torch.eq(inputter,outputter)):
            correct+=1
            correct_chars+=idx
            tot_chars+=idx
        else:
            correct_chars+=torch.sum(inputter == outputter).item()
            tot_chars+=idx
            
#         print(correct,correct_chars,tot_chars)
        
    return correct.item(),correct_chars.item(),tot_chars.item()
            
            
        
    

In [20]:
accuracy_calc(ss1['output'],ss1['output'])

(16.0, 134.0, 134.0)

In [34]:
def train(seq2seq, iterator, optimizer, criterion):
    
    
    seq2seq.train()
    
    epoch_loss = 0
    correct=0
    correct_char=0
    tot_char=0
    
    relax_acc=0
    
    
    for batch in iterator:
        optimizer.zero_grad()
        outputs = seq2seq(batch['input'], batch['output'])
        batch_label=batch['output'].transpose(0,1)
        batch_size=len(batch['output'])
        
        _, predicted = torch.max(outputs, dim=2)
        outputs_flatten = outputs.view(-1, outputs.shape[-1])
        trg_flatten = batch_label.reshape(-1)
        

        trg_flatten.requires_grad=False
        loss = criterion(outputs_flatten, trg_flatten)
        correct_temp,correct_chars_temp,tot_chars_temp=accuracy_calc(batch['output'],predicted.transpose(0,1))
        
        #___________
        
        correct+=correct_temp
        correct_char+=correct_chars_temp
        tot_char+=tot_chars_temp
        
        
        #_______________
        

        loss.backward()
        optimizer.step()
        

        epoch_loss += loss.item()
        


    return epoch_loss / len(iterator), correct/(len(iterator)*16),correct_char/tot_char

In [35]:
def evaluate(seq2seq, iterator, criterion):
    seq2seq.eval()

    epoch_loss = 0
    correct=0
    correct_char=0
    tot_char=0
    
    relax_acc=0
    
    with torch.no_grad():
        for batch in iterator:
            outputs = seq2seq(batch['input'], batch['output'],teacher_forcing_ratio=0)
            batch_label=batch['output'].transpose(0,1)
            batch_size=len(batch['output'])


            _, predicted = torch.max(outputs, dim=2)
            #print('wow_preds',predicted.shape)

            outputs_flatten = outputs.view(-1, outputs.shape[-1])
            trg_flatten = batch_label.reshape(-1)

            loss = criterion(outputs_flatten, trg_flatten)
            
            correct_temp,correct_chars_temp,tot_chars_temp=accuracy_calc(batch['output'],predicted.transpose(0,1))
        
            #___________

            correct+=correct_temp
            correct_char+=correct_chars_temp
            tot_char+=tot_chars_temp

            #_______________       
            
            epoch_loss += loss.item()
            

    return epoch_loss / len(iterator), correct/(len(iterator)*16),correct_char/tot_char



In [36]:
def epoch_time(start_time, end_time):
    e_time = end_time - start_time
    mins = e_time // 60
    secs = e_time%60
    return mins, secs,

In [37]:
def count_params(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)

In [38]:
import time
import random

In [39]:
import torch.optim as optim
def make_model(train_iterator,valid_iterator,N_EPOCHS=20):
    E=Encoder(30,128,128,2,'lstm',0.27,False)
    E=E.to(device)
    D=Decoder(68,128,128,2,'lstm',0.27,False)
    D=D.to(device)
    S=Seq2Seq(E,D,device)
    S.to(device)    
    print(f'The model has {count_params(S):,} trainable parameters')
    
    optimizer = optim.Adam(S.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=hindi_alphabet_to_index['.'])
    criterion=criterion.to(device)
    
    best_valid_loss = float('inf')
#     return S

    for epoch in range(N_EPOCHS):    
        start_time = time.time()
        train_loss,train_acc,train_stuff = train(S, train_iterator, optimizer, criterion)
        valid_loss,valid_acc,val_stuff = evaluate(S, valid_iterator, criterion)
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(S.state_dict(), 'model1.pt')

        # it's easier to see a change in perplexity between epoch as it's an exponential
        # of the loss, hence the scale of the measure is much bigger
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s')
        print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        print(f'\t Relaxed Train. Acc: {train_stuff*100:.2f}% | Relaxed Val. Acc: {val_stuff*100:.2f}%')
        
    return S

    

In [40]:
train_iterator=train_dataloader
valid_iterator=val_dataloader
SS=make_model(train_iterator,valid_iterator,N_EPOCHS=15)

The model has 549,700 trainable parameters
Epoch: 01 | Time: 0.0m 33.43s
	 Train Loss: 2.676 | Train Acc: 0.37%
	 Val. Loss: 2.045 |  Val. Acc: 7.71%
	 Relaxed Train. Acc: 33.89% | Relaxed Val. Acc: 52.78%
Epoch: 02 | Time: 0.0m 33.38s
	 Train Loss: 1.644 | Train Acc: 6.91%
	 Val. Loss: 1.657 |  Val. Acc: 19.09%
	 Relaxed Train. Acc: 60.68% | Relaxed Val. Acc: 63.59%
Epoch: 03 | Time: 0.0m 33.33s
	 Train Loss: 1.393 | Train Acc: 13.36%
	 Val. Loss: 1.602 |  Val. Acc: 25.00%
	 Relaxed Train. Acc: 68.23% | Relaxed Val. Acc: 66.11%
Epoch: 04 | Time: 0.0m 33.29s
	 Train Loss: 1.289 | Train Acc: 17.07%
	 Val. Loss: 1.529 |  Val. Acc: 26.27%
	 Relaxed Train. Acc: 71.49% | Relaxed Val. Acc: 68.20%
Epoch: 05 | Time: 0.0m 33.24s
	 Train Loss: 1.224 | Train Acc: 19.82%
	 Val. Loss: 1.510 |  Val. Acc: 27.93%
	 Relaxed Train. Acc: 73.64% | Relaxed Val. Acc: 69.80%
Epoch: 06 | Time: 0.0m 33.23s
	 Train Loss: 1.173 | Train Acc: 22.12%
	 Val. Loss: 1.497 |  Val. Acc: 28.69%
	 Relaxed Train. Acc: 75.3

In [45]:
a=next(iter(train_dataloader))

In [46]:
bya=SS(a['input'],a['output'])

In [47]:
a['output'].shape

torch.Size([16, 30])

In [48]:
bya.shape

torch.Size([30, 16, 68])

In [49]:
import math

In [50]:
def word_from_torchies(torchie1,index_toalp):
    torchie=torchie1.cpu().numpy()
    return word_from_vecs(torchie,index_toalp,False)

In [51]:
def word_from_batch(batch):
    wordlet=[]
    for i in range(len(batch)):
        wordlet.append(word_from_torchies(batch[i],index_to_hindi_alphabet))
    return wordlet
        

In [64]:
word_from_torchies(bya.transpose(0,1).argmax(2)[0],index_to_hindi_alphabet)

'मेडियाच'

In [57]:
index_to_hindi_alphabet[65]

'ङ'

In [58]:
def make_wrd(stuff):
    asa=[]
    for k in stuff.cpu().numpy():
        asa.append(index_to_hindi_alphabet[k])
    return "".join(asa)
    

In [59]:
make_wrd(bya.transpose(0,1).argmax(2)[10])

'<दोशारोपत्र>>>>>>>>>>>ी>>>>>>>'

In [67]:
bya.transpose(0,1).argmax(2)[10]

tensor([ 0,  7, 17, 25,  9, 11, 17, 18, 14,  6, 11,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1, 20,  1,  1,  1,  1,  1,  1,  1], device='cuda:0')

In [68]:
for i in range(16):
    print('................')
    print(i)
    print(word_from_torchies(a['output'][i],index_to_hindi_alphabet),\
         '---',\
         make_wrd(bya.transpose(0,1).argmax(2)[i])\
         )


................
0
मीडियाच --- <मेडियाच>>>>>>>आआई>>>>>>>>>>>>
................
1
जशिला --- <जशिला>>>>>>>>>>>>>>>>>>>>>>>>
................
2
डिजाइनर्स --- <डिजिइनर्स>>>>>>>>>>>>>>>>>>>>
................
3
लाइसराम --- <लाससराम>>>>>र>>>>>>>>>>>>>>>>
................
4
इच्छेला --- <इछेछेला>>>>>>>>>>>>>>>>>>>>>>
................
5
गँवाना --- <गंवनना>>>>>>>>>>>>>>>>>>>>>>>
................
6
क्रोधाग्नी --- <क्रोधगग्नी>>ी>ीी>>>>>>>>>>>>>
................
7
रस्सियाँ --- <रसससियाँ>>>>>>>>>>>ीी>>>>>>>>
................
8
सनाचा --- <सााचा>>>>>>>>>>>>>>>>>>>>>>>>
................
9
पटरीरहित --- <पताीरहित>>>>ी>>>>>>ीी>>>>>>>>
................
10
दोषारोपत्र --- <दोशारोपत्र>>>>>>>>>>>ी>>>>>>>
................
11
यदविन्दर --- <यद्िं्द्र>>>>>>>>>>ीी>>>>>>>>
................
12
मान्छू --- <मांचचू>>>>>>>>>>>>>>>>>>>>>>>
................
13
डायमंडचा --- <डायमोडचा>>>>>>>>>>>ीी>>>>>>>>
................
14
शिवारचे --- <शिवारचे>>>>>>>>>>>>>>>>>>>>>>
................
15
स्प्रेस --- <स्प्रेस>>>>>>>>>>>>>>>>>>>

In [ ]:
word (self, source_batch,target_batch):
    max_len, batch_size = target_batch.shape
    outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)


            # last hidden & cell state of the encoder is used as the decoder's initial hidden state
    hidden, cell = self.encoder(source_batch)

    wordet=[]


    trg = torch.tensor(hindi_alphabet_to_index['<'])
    trg=trg.to(device)
    wordet.append(index_to_hindi_alphabet(trg.cpu().numpy()))
    for i in range(1, max_len):
        prediction, hidden, cell = self.decoder(trg, hidden, cell)
        outputs[i] = prediction
        trg = prediction.argmax(1)
        wordet.append(index_to_hindi_alphabet(trg.cpu().numpy()))


    return ''.join(wordet)

In [ ]:
word_from_vecs(X_valid[0],index_to_english_alphabet,False)

In [ ]:
word_from_vecs(y_valid[0],index_to_hindi_alphabet,False)

In [ ]:
next(iter(train_dataloader))

In [ ]:
            for j in range(predicted.shape[1]):
                predicted_seq = predicted[:, j]
                targets_seq = target_seq[:, j]

                # Find the index of the first EOS token in the sequence
                eos_idx = (targets_seq == hin_token_map["\n"]).nonzero()
                if eos_idx.numel() > 0:
                    eos_idx = eos_idx[0][0]
                    predicted_seq = predicted_seq[:eos_idx]
                    targets_seq = targets_seq[:eos_idx]

In [ ]:
s